On commence d'abord par traduire les phrases du francais en angalis

In [ ]:
from mtranslate import translate
import time
import concurrent.futures

number_threads = 4
progress_file_path = '/content/drive/MyDrive/Colab Notebooks/progress.txt'

def load_progress():
    try:
        with open(progress_file_path, 'r') as progress_file:
            return int(progress_file.read().strip())
    except FileNotFoundError:
        return 0

def save_progress(line_number):
    with open(progress_file_path, 'w') as progress_file:
        progress_file.write(str(line_number))

current_progress = load_progress()

with open('/content/drive/MyDrive/Colab Notebooks/merged-output.txt', 'r') as f:
    with open('/content/drive/MyDrive/Colab Notebooks/1.txt', 'a') as out:
        while True:
            with concurrent.futures.ThreadPoolExecutor(max_workers=number_threads) as executor:
                future_to_line = {}
                line_number = 0

                for line in f:
                    line_number += 1

                    if line_number <= current_progress:
                        continue

                    future = executor.submit(translate, line.strip(), "english")
                    future_to_line[future] = (line_number, line)

                for future in concurrent.futures.as_completed(future_to_line):
                    line_number, line = future_to_line[future]
                    try:
                        translated_line = future.result()
                        out.write(translated_line + '\n')
                        current_progress = line_number
                        save_progress(current_progress)
                    except Exception as e:
                        print(f"Encountered an exception: {e}. Saving progress and stopping script...")
                        save_progress(current_progress)
                        exit(1)

                if line_number == current_progress:
                    break


Désormais, il est possible de vérifier les phrases et de ne conserver que celles qui sont grammaticalement correctes.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def check_grammar(sentence):
    doc = nlp(sentence)

    
    if len(doc) < 2:
        return "Incomplete sentence"
    for sent in doc.sents:
        if len(sent) < 2:
            return "Fragmented sentence"
    
    for token in doc:
        if token.dep_ == 'nsubj' and not token.is_punct:

            break
    else:
        return "Missing subject"
    
    for token in doc:
        if token.pos_ == 'VERB' and not token.is_punct:
            break
    else:
        return "Missing verb"
    
    for token in doc:
        if token.pos_ == 'VERB' and token.dep_ == 'aux' and not token.is_punct:
            return "Auxiliary verb should not be used"
        if token.tag_ in ['VBD', 'VBN'] and not token.is_punct:
            return "Use present tense instead of past participle or past tense"
        if token.dep_ == 'nsubjpass' and not token.is_punct:
            return "Passive voice should not be used"
        if token.tag_ == 'NEG' and not token.is_punct:
            return "Negative sentence"
    
    return None

with open('/content/drive/MyDrive/Colab Notebooks/1.txt', 'r') as input_file, open('/content/drive/MyDrive/Colab Notebooks/2.txt', 'w') as output_file:
    for line in input_file:
        doc = nlp(line)
        for sent in doc.sents:
            result = check_grammar(sent.text)
            if not result:
                output_file.write(sent.text + '\n')


Une fois cette étape terminée, les phrases sont traduites de nouveau en français.


In [ ]:
from mtranslate import translate
import time
import concurrent.futures

number_threads = 4
progress_file_path = '/content/drive/MyDrive/Colab Notebooks/progress.txt'

def load_progress():
    try:
        with open(progress_file_path, 'r') as progress_file:
            return int(progress_file.read().strip())
    except FileNotFoundError:
        return 0

def save_progress(line_number):
    with open(progress_file_path, 'w') as progress_file:
        progress_file.write(str(line_number))

current_progress = load_progress()

with open('/content/drive/MyDrive/Colab Notebooks/2.txt', 'r') as f:
    with open('/content/drive/MyDrive/Colab Notebooks/3.txt', 'a') as out:
        while True:
            with concurrent.futures.ThreadPoolExecutor(max_workers=number_threads) as executor:
                future_to_line = {}
                line_number = 0

                for line in f:
                    line_number += 1

                    if line_number <= current_progress:
                        continue

                    future = executor.submit(translate, line.strip(), "fr","en")
                    future_to_line[future] = (line_number, line)

                for future in concurrent.futures.as_completed(future_to_line):
                    line_number, line = future_to_line[future]
                    try:
                        translated_line = future.result()
                        out.write(translated_line + '\n')
                        current_progress = line_number
                        save_progress(current_progress)
                    except Exception as e:
                        print(f"Encountered an exception: {e}. Saving progress and stopping script...")
                        save_progress(current_progress)
                        exit(1)

                if line_number == current_progress:
                    break


Enfin, un fichier Excel est créé avec les phrases simples pour une meilleure visualisation. Les phrases complexes seront ajoutées dans les mêmes lignes ultérieurement.

In [ ]:
import openpyxl

with open('/content/drive/MyDrive/Colab Notebooks/3.txt', 'r') as f:
    sentences = [line.strip() for line in f if line.strip()]

workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = 'Sheet1'

worksheet.cell(row=1, column=1).value = 'A'

for index, sentence in enumerate(sentences, start=2):
    worksheet.cell(row=index, column=1).value = sentence

workbook.save('/content/drive/MyDrive/Colab Notebooks/simple_french.xlsx')
